In [1]:
import numpy as np
import pandas as pd

# plot
import matplotlib.pyplot as plt
import seaborn as sns
import re
import scipy
from collections import Counter
%matplotlib inline
sns.set_style("darkgrid")

#sklearn
from sklearn.model_selection import train_test_split

# For the network
import glob
import cv2
import os

import ipywidgets as iw
from IPython.display import display, clear_output

!pip install tensorflow==2.0.0-alpha0

import tensorflow.keras
from tensorflow.keras import layers, models, optimizers, metrics 


#from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
#tf.enable_eager_execution()

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


# Import various componenets for model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop

# Import the backend
from tensorflow.python.keras import backend as k

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import warnings
# filter warnings
warnings.filterwarnings('ignore')

In [2]:
traffic_18_m = pd.read_csv('traffic_18_m.csv')

In [3]:
traffic_18_m['speed_index'] = 0

# Minus
traffic_18_m['speed_index'].loc[(traffic_18_m['SPEED']<=(traffic_18_m['speed_limit']-10))&
                                  (traffic_18_m['SPEED']>(traffic_18_m['speed_limit']-20))] = -10
traffic_18_m['speed_index'].loc[(traffic_18_m['SPEED']<=(traffic_18_m['speed_limit']-20))] = -20


# Minus
traffic_18_m['speed_index'].loc[(traffic_18_m['SPEED']>=(traffic_18_m['speed_limit']+10))&
                                  (traffic_18_m['SPEED']<(traffic_18_m['speed_limit']+20))] = 10
traffic_18_m['speed_index'].loc[(traffic_18_m['SPEED']>=(traffic_18_m['speed_limit']+20))] = 20

traffic_18_m['speed_index'].loc[traffic_18_m['SPEED']==0] = -100



#traffic_18_m['speed_index'] = traffic_18_m['speed_index'].astype('category')

In [4]:
traffic_s = traffic_18_m.sample(frac=1, random_state=1)

traffic_cnn = pd.DataFrame(columns=traffic_s.columns)

for i in traffic_s.speed_index.unique():
    label = traffic_s.loc[traffic_s['speed_index']==i][:500000]
    traffic_cnn = pd.concat([traffic_cnn, label])
    
traffic_cnn.speed_index.value_counts()

-10     500000
-20     500000
-100    500000
 20     500000
 10     500000
 0      500000
Name: speed_index, dtype: int64

In [5]:
traffic_cnn.dtypes

Unnamed: 0                 object
index                      object
ID                         object
SPEED                     float64
TRAVEL_TIME                object
STATUS                     object
DATA_AS_OF                 object
LINK_ID                    object
LINK_POINTS                object
ENCODED_POLY_LINE          object
ENCODED_POLY_LINE_LVLS     object
OWNER                      object
TRANSCOM_ID                object
BOROUGH                    object
LINK_NAME                  object
year                       object
month                      object
dayofweek                  object
RecordedAtTime             object
hour                       object
minute                     object
poly_num                   object
LINK_START                 object
LINK_END                   object
LINK_MIDDLE                object
speed_id                   object
speed_limit                object
lion_id                    object
Street                     object
FeatureTyp    

In [6]:
traffic_cnn = traffic_cnn.drop(columns=['Unnamed: 0','index','LINK_POINTS','ENCODED_POLY_LINE','ENCODED_POLY_LINE_LVLS','TRANSCOM_ID','Join_ID'])

In [7]:
tonumeric = ['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour','minute','poly_num','BikeLane', 
             'weekend','morn_rush_hr', 'eve_rush_hr','Number_Tot','Number_Tra','SeqNum','StreetCode','lion_id',
             'speed_id','speed_limit']
tocategory = ['Snow_Prior','NonPed','RB_Layer','SegmentTyp','FeatureTyp','Street','BOROUGH']

# turn columns into numberic
for i in tonumeric:
    traffic_cnn[i] = pd.to_numeric(traffic_cnn[i])

# to a category then immediately into a coded column
for i in tocategory:
    traffic_cnn[i] = traffic_cnn[i].astype('category')
    traffic_cnn[i+'_codes'] = traffic_cnn[i].cat.codes

In [8]:
traffic_cnn = traffic_cnn.drop(columns='NonPed')
traffic_cnn = traffic_cnn.dropna()

In [9]:
traffic_cnn = traffic_cnn.drop(['DATA_AS_OF','OWNER', 'BOROUGH','LINK_NAME','RecordedAtTime',
                                'LINK_START','LINK_END','LINK_MIDDLE','Street', 'FeatureTyp', 'SegmentTyp', 
                                'RB_Layer', 'TrafDir','Snow_Prior', 'TRAVEL_TIME','SPEED'], 1)

In [10]:
#traffic_cnn['speed_index'] = pd.to_numeric(traffic_cnn['speed_index'])

In [11]:
traffic_cnn.dtypes

ID                        int64
STATUS                    int64
LINK_ID                   int64
year                      int64
month                     int64
dayofweek                 int64
hour                      int64
minute                    int64
poly_num                  int64
speed_id                  int64
speed_limit               int64
lion_id                   int64
SeqNum                    int64
StreetCode                int64
StreetWidt              float64
BikeLane                  int64
Number_Tra                int64
Number_Tot                int64
weekend                   int64
morn_rush_hr              int64
eve_rush_hr               int64
morning_rush_avg_spd    float64
evening_rush_avg_spd    float64
wknd_avg_spd            float64
overall_avg_spd         float64
overall_std_speed       float64
speed_index              object
Snow_Prior_codes           int8
NonPed_codes               int8
RB_Layer_codes             int8
SegmentTyp_codes           int8
FeatureT

## Deep Neural Net

In [12]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('speed_index')                     
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [13]:
traffic_cnn.shape

(3000000, 34)

In [14]:
# to split a training and test sample
traffic_train, traffic_test = train_test_split(traffic_cnn, test_size=0.5, random_state=0)
traffic_test, traffic_val = train_test_split(traffic_test, test_size=0.5, random_state=0)

In [15]:
print(traffic_train.shape)
print(traffic_test.shape)
print(traffic_val.shape)

(1500000, 34)
(750000, 34)
(750000, 34)


In [28]:
batch_size = 32
train_ds = df_to_dataset(traffic_train, batch_size=batch_size)
val_ds = df_to_dataset(traffic_val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(traffic_test, shuffle=False, batch_size=batch_size)

In [17]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of Street Widths:', feature_batch['StreetWidt'])
    print('A batch of targets:', label_batch )

Every feature: ['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour', 'minute', 'poly_num', 'speed_id', 'speed_limit', 'lion_id', 'SeqNum', 'StreetCode', 'StreetWidt', 'BikeLane', 'Number_Tra', 'Number_Tot', 'weekend', 'morn_rush_hr', 'eve_rush_hr', 'morning_rush_avg_spd', 'evening_rush_avg_spd', 'wknd_avg_spd', 'overall_avg_spd', 'overall_std_speed', 'Snow_Prior_codes', 'NonPed_codes', 'RB_Layer_codes', 'SegmentTyp_codes', 'FeatureTyp_codes', 'Street_codes', 'BOROUGH_codes']
A batch of Street Widths: tf.Tensor(
[34. 32. 26. 36. 60. 34. 30. 24. 30. 80. 32. 32. 36. 25. 32. 34. 40. 36.
 26. 36. 18. 20. 32. 40. 32. 20. 20. 30. 25. 36. 32. 30. 32. 32. 20. 24.
 42. 60. 42. 20. 42. 36. 60. 36. 18. 32. 30. 30. 20. 40. 38. 20. 36. 34.
 20. 33. 33. 33. 42. 25. 33. 32. 26. 20.], shape=(64,), dtype=float32)
A batch of targets: tf.Tensor(
[  10    0 -100   20   10   10    0   10  -20    0   10  -10  -10    0
  -10   20   20  -10  -10  -20   20 -100  -20  -20   10 -100   10  -20
   10   

In [18]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [19]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [20]:
traffic_cnn.columns

Index(['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour',
       'minute', 'poly_num', 'speed_id', 'speed_limit', 'lion_id', 'SeqNum',
       'StreetCode', 'StreetWidt', 'BikeLane', 'Number_Tra', 'Number_Tot',
       'weekend', 'morn_rush_hr', 'eve_rush_hr', 'morning_rush_avg_spd',
       'evening_rush_avg_spd', 'wknd_avg_spd', 'overall_avg_spd',
       'overall_std_speed', 'speed_index', 'Snow_Prior_codes', 'NonPed_codes',
       'RB_Layer_codes', 'SegmentTyp_codes', 'FeatureTyp_codes',
       'Street_codes', 'BOROUGH_codes'],
      dtype='object')

In [21]:
hour = feature_column.numeric_column('hour')
demo(hour)

W0528 23:29:08.985878 4616742336 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/feature_column/feature_column_v2.py:2758: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


[[ 3.]
 [ 3.]
 [11.]
 [ 3.]
 [ 7.]
 [14.]
 [ 2.]
 [ 3.]
 [14.]
 [ 7.]
 [ 6.]
 [16.]
 [20.]
 [ 2.]
 [ 8.]
 [15.]
 [ 1.]
 [ 8.]
 [16.]
 [16.]
 [ 9.]
 [17.]
 [23.]
 [22.]
 [14.]
 [23.]
 [ 5.]
 [11.]
 [14.]
 [14.]
 [19.]
 [16.]
 [22.]
 [ 7.]
 [16.]
 [ 4.]
 [ 5.]
 [13.]
 [ 5.]
 [14.]
 [ 0.]
 [19.]
 [14.]
 [13.]
 [17.]
 [13.]
 [13.]
 [ 0.]
 [ 1.]
 [22.]
 [19.]
 [10.]
 [ 0.]
 [19.]
 [17.]
 [ 5.]
 [23.]
 [ 5.]
 [ 9.]
 [17.]
 [ 1.]
 [ 2.]
 [16.]
 [ 7.]]


In [22]:
len(list(feature_batch.keys()))

33

In [23]:
traffic_cnn.columns

Index(['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour',
       'minute', 'poly_num', 'speed_id', 'speed_limit', 'lion_id', 'SeqNum',
       'StreetCode', 'StreetWidt', 'BikeLane', 'Number_Tra', 'Number_Tot',
       'weekend', 'morn_rush_hr', 'eve_rush_hr', 'morning_rush_avg_spd',
       'evening_rush_avg_spd', 'wknd_avg_spd', 'overall_avg_spd',
       'overall_std_speed', 'speed_index', 'Snow_Prior_codes', 'NonPed_codes',
       'RB_Layer_codes', 'SegmentTyp_codes', 'FeatureTyp_codes',
       'Street_codes', 'BOROUGH_codes'],
      dtype='object')

In [30]:
traffic_cnn['speed_limit'].unique()

array([50, 40, 25, 35, 45,  0])

In [31]:
feature_columns = []

# numeric cols
for header in ['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour',
               'minute', 'poly_num', 'speed_id', 'speed_limit', 'lion_id', 'SeqNum',
               'StreetCode', 'StreetWidt', 'BikeLane', 'Number_Tra', 'Number_Tot',
               'weekend', 'morn_rush_hr', 'eve_rush_hr', 'morning_rush_avg_spd',
               'evening_rush_avg_spd', 'wknd_avg_spd', 'overall_avg_spd',
               'overall_std_speed', 'Snow_Prior_codes', 'NonPed_codes',
               'RB_Layer_codes', 'SegmentTyp_codes', 'FeatureTyp_codes',
               'Street_codes', 'BOROUGH_codes']:
    feature_columns.append(feature_column.numeric_column(header))

# indicator cols
limit = feature_column.categorical_column_with_vocabulary_list(
      'speed_limit', traffic_cnn['speed_limit'].unique())
limit_one_hot = feature_column.indicator_column(limit)
feature_columns.append(limit_one_hot)
    
# embedding cols
limit_embedding = feature_column.embedding_column(limit, dimension=8)
feature_columns.append(limit_embedding)


# crossed cols
hour_status = feature_column.crossed_column(['hour', 'STATUS'], hash_bucket_size=1000)
hour_status = feature_column.indicator_column(hour_status)
feature_columns.append(hour_status)

In [32]:
feature_columns[6]

NumericColumn(key='hour', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [33]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [35]:
from tensorflow.keras.layers import Dense, Dropout

model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu', input_dim=34),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=25)

Epoch 1/25
46875/46875 [==============================] - 3766s 80ms/step - loss: -244.3378 - accuracy: 0.1509 - val_loss: -257.3061 - val_accuracy: 0.1658
Epoch 2/25
46875/46875 [==============================] - 4302s 92ms/step - loss: -256.5647 - accuracy: 0.1670 - val_loss: -257.3061 - val_accuracy: 0.1658
Epoch 3/25
46875/46875 [==============================] - 5961s 127ms/step - loss: -256.5647 - accuracy: 0.1670 - val_loss: -257.3061 - val_accuracy: 0.1658
Epoch 4/25
46875/46875 [==============================] - 7928s 169ms/step - loss: -256.5549 - accuracy: 0.1670 - val_loss: -257.3061 - val_accuracy: 0.1658
Epoch 5/25
12792/46875 [=======>......................] - ETA: 1:49:32 - loss: -256.7819 - accuracy: 0.1670

KeyboardInterrupt: 

___

## LSTM Attempt 1

In [16]:
traffic_18_m = pd.read_csv('traffic_18_m.csv')

In [17]:
traffic_18_m = traffic_18_m.drop(['Unnamed: 0', 'index'],1)

In [26]:
traffic_18_m['DATA_AS_OF'] = pd.DatetimeIndex(traffic_18_m['DATA_AS_OF'])

traffic_sample = traffic_18_m.loc[(traffic_18_m['DATA_AS_OF']>'2018-5-13')&(traffic_18_m['DATA_AS_OF']<'2018-5-20')]

traffic_sample = traffic_sample.drop(columns=['LINK_POINTS','ENCODED_POLY_LINE','ENCODED_POLY_LINE_LVLS','TRANSCOM_ID','Join_ID'])

In [27]:
traffic_sample.shape

(250191, 44)

In [28]:
tonumeric = ['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour','minute','poly_num','BikeLane', 
             'weekend','morn_rush_hr', 'eve_rush_hr','Number_Tot','Number_Tra','SeqNum','StreetCode','lion_id',
             'speed_id','speed_limit']
tocategory = ['Snow_Prior','NonPed','RB_Layer','SegmentTyp','FeatureTyp','Street','BOROUGH']

# turn columns into numberic
for i in tonumeric:
    traffic_sample[i] = pd.to_numeric(traffic_sample[i])

# to a category then immediately into a coded column
for i in tocategory:
    traffic_sample[i] = traffic_sample[i].astype('category')
    traffic_sample[i+'_codes'] = traffic_sample[i].cat.codes

In [29]:
traffic_sample.columns

Index(['ID', 'SPEED', 'TRAVEL_TIME', 'STATUS', 'DATA_AS_OF', 'LINK_ID',
       'OWNER', 'BOROUGH', 'LINK_NAME', 'year', 'month', 'dayofweek',
       'RecordedAtTime', 'hour', 'minute', 'poly_num', 'LINK_START',
       'LINK_END', 'LINK_MIDDLE', 'speed_id', 'speed_limit', 'lion_id',
       'Street', 'FeatureTyp', 'SegmentTyp', 'RB_Layer', 'NonPed', 'TrafDir',
       'SeqNum', 'StreetCode', 'StreetWidt', 'BikeLane', 'Snow_Prior',
       'Number_Tra', 'Number_Tot', 'weekend', 'morn_rush_hr', 'eve_rush_hr',
       'morning_rush_avg_spd', 'evening_rush_avg_spd', 'wknd_avg_spd',
       'overall_avg_spd', 'overall_std_speed', 'speed_index',
       'Snow_Prior_codes', 'NonPed_codes', 'RB_Layer_codes',
       'SegmentTyp_codes', 'FeatureTyp_codes', 'Street_codes',
       'BOROUGH_codes'],
      dtype='object')

In [30]:
traffic_sample = traffic_sample.drop(['speed_index','OWNER', 'BOROUGH','LINK_NAME','RecordedAtTime',
                                      'LINK_START','LINK_END','LINK_MIDDLE','Street', 'FeatureTyp', 'SegmentTyp', 
                                      'RB_Layer', 'TrafDir','Snow_Prior', 'TRAVEL_TIME','NonPed'], 1)

In [34]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [36]:
# fix random seed for reproducibility
np.random.seed(7)

In [37]:
dataset = traffic_sample.values

In [38]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [39]:
# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

167627 82564


In [42]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [86]:
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [87]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [98]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(64, input_shape=(1, look_back)))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, epochs=25, batch_size=20, validation_data=(testX, testY))

Train on 167623 samples, validate on 82560 samples
Epoch 1/25
167623/167623 [==============================] - 17s 101us/sample - loss: 0.0044 - accuracy: 0.0230 - val_loss: 1.3061e-04 - val_accuracy: 0.0000e+00
Epoch 2/25
167623/167623 [==============================] - 17s 101us/sample - loss: 0.0013 - accuracy: 0.0231 - val_loss: 1.3227e-04 - val_accuracy: 0.0000e+00
Epoch 3/25
167623/167623 [==============================] - 17s 101us/sample - loss: 0.0013 - accuracy: 0.0231 - val_loss: 9.1078e-05 - val_accuracy: 0.0000e+00
Epoch 4/25
167623/167623 [==============================] - 16s 96us/sample - loss: 0.0013 - accuracy: 0.0231 - val_loss: 6.8509e-05 - val_accuracy: 0.0000e+00
Epoch 5/25
167623/167623 [==============================] - 16s 97us/sample - loss: 0.0013 - accuracy: 0.0231 - val_loss: 9.7752e-05 - val_accuracy: 0.0000e+00
Epoch 6/25
167623/167623 [==============================] - 17s 100us/sample - loss: 0.0013 - accuracy: 0.0231 - val_loss: 8.1630e-05 - val_accura

## LSTM Attempt 2

In [36]:
traffic_18_m['DATA_AS_OF'] = pd.DatetimeIndex(traffic_18_m['DATA_AS_OF'])

traffic_sample = traffic_18_m.loc[(traffic_18_m['DATA_AS_OF']>'2018-5-13')&(traffic_18_m['DATA_AS_OF']<'2018-5-20')]

traffic_sample = traffic_sample.drop(columns=['LINK_POINTS','ENCODED_POLY_LINE','ENCODED_POLY_LINE_LVLS','TRANSCOM_ID','Join_ID'])



In [37]:
traffic_sample.shape

(250191, 46)

In [38]:
tonumeric = ['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour','minute','poly_num','BikeLane', 
             'weekend','morn_rush_hr', 'eve_rush_hr','Number_Tot','Number_Tra','SeqNum','StreetCode','lion_id',
             'speed_id','speed_limit']
tocategory = ['Snow_Prior','NonPed','RB_Layer','SegmentTyp','FeatureTyp','Street','BOROUGH']

# turn columns into numberic
for i in tonumeric:
    traffic_sample[i] = pd.to_numeric(traffic_sample[i])

# to a category then immediately into a coded column
for i in tocategory:
    traffic_sample[i] = traffic_sample[i].astype('category')
    traffic_sample[i+'_codes'] = traffic_sample[i].cat.codes

In [39]:
traffic_sample.columns

Index(['Unnamed: 0', 'index', 'ID', 'SPEED', 'TRAVEL_TIME', 'STATUS',
       'DATA_AS_OF', 'LINK_ID', 'OWNER', 'BOROUGH', 'LINK_NAME', 'year',
       'month', 'dayofweek', 'RecordedAtTime', 'hour', 'minute', 'poly_num',
       'LINK_START', 'LINK_END', 'LINK_MIDDLE', 'speed_id', 'speed_limit',
       'lion_id', 'Street', 'FeatureTyp', 'SegmentTyp', 'RB_Layer', 'NonPed',
       'TrafDir', 'SeqNum', 'StreetCode', 'StreetWidt', 'BikeLane',
       'Snow_Prior', 'Number_Tra', 'Number_Tot', 'weekend', 'morn_rush_hr',
       'eve_rush_hr', 'morning_rush_avg_spd', 'evening_rush_avg_spd',
       'wknd_avg_spd', 'overall_avg_spd', 'overall_std_speed', 'speed_index',
       'Snow_Prior_codes', 'NonPed_codes', 'RB_Layer_codes',
       'SegmentTyp_codes', 'FeatureTyp_codes', 'Street_codes',
       'BOROUGH_codes'],
      dtype='object')

In [40]:
traffic_sample = traffic_sample.drop(['Unnamed: 0', 'index','speed_index','OWNER', 'BOROUGH','LINK_NAME', 'RecordedAtTime',
                                      'LINK_START','LINK_END','LINK_MIDDLE','Street', 'FeatureTyp', 'SegmentTyp', 
                                      'RB_Layer', 'TrafDir','Snow_Prior', 'TRAVEL_TIME','NonPed'], 1)

In [41]:
traffic_sample.index = traffic_sample['DATA_AS_OF']

In [42]:
traffic_sample = traffic_sample.drop(['DATA_AS_OF'], 1)

In [43]:
traffic_sample.head()

,ID,SPEED,STATUS,LINK_ID,year,month,dayofweek,hour,minute,poly_num,...,wknd_avg_spd,overall_avg_spd,overall_std_speed,Snow_Prior_codes,NonPed_codes,RB_Layer_codes,SegmentTyp_codes,FeatureTyp_codes,Street_codes,BOROUGH_codes
DATA_AS_OF,,,,,,,,,,,,,,,,,,,,,
2018-05-19 23:58:09,423,47.84,0,4616299,2018,5,5,23,58,44,...,49.801891,46.591828,11.256594,0,1,1,1,0,26,3
2018-05-19 23:53:09,423,47.22,0,4616299,2018,5,5,23,53,44,...,49.801891,46.591828,11.256594,0,1,1,1,0,26,3
2018-05-19 23:48:09,423,43.49,0,4616299,2018,5,5,23,48,44,...,49.801891,46.591828,11.256594,0,1,1,1,0,26,3
2018-05-19 23:43:09,423,37.90,0,4616299,2018,5,5,23,43,44,...,49.801891,46.591828,11.256594,0,1,1,1,0,26,3
2018-05-19 23:38:09,423,37.28,0,4616299,2018,5,5,23,38,44,...,49.801891,46.591828,11.256594,0,1,1,1,0,26,3


In [45]:
traffic_sample.columns

Index(['ID', 'SPEED', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek',
       'hour', 'minute', 'poly_num', 'speed_id', 'speed_limit', 'lion_id',
       'SeqNum', 'StreetCode', 'StreetWidt', 'BikeLane', 'Number_Tra',
       'Number_Tot', 'weekend', 'morn_rush_hr', 'eve_rush_hr',
       'morning_rush_avg_spd', 'evening_rush_avg_spd', 'wknd_avg_spd',
       'overall_avg_spd', 'overall_std_speed', 'Snow_Prior_codes',
       'NonPed_codes', 'RB_Layer_codes', 'SegmentTyp_codes',
       'FeatureTyp_codes', 'Street_codes', 'BOROUGH_codes'],
      dtype='object')

In [54]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# load dataset
dataset = traffic_sample
values = dataset.values
# integer encode direction
encoder = preprocessing.LabelEncoder()
values[:,1] = encoder.fit_transform(values[:,1])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
#reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
reframed.shape

(250190, 68)

In [55]:
labels = traffic_sample.reset_index()
reframed['speed'] = labels['SPEED']

In [56]:
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var26(t),var27(t),var28(t),var29(t),var30(t),var31(t),var32(t),var33(t),var34(t),speed
1,0.933628,0.631148,1.0,0.66061,0.0,0.0,0.833333,1.0,0.982759,0.677419,...,0.790037,0.367649,0.0,1.0,1.0,0.25,0.0,0.866667,0.6,47.22
2,0.933628,0.622951,1.0,0.66061,0.0,0.0,0.833333,1.0,0.896552,0.677419,...,0.790037,0.367649,0.0,1.0,1.0,0.25,0.0,0.866667,0.6,43.49
3,0.933628,0.573770,1.0,0.66061,0.0,0.0,0.833333,1.0,0.810345,0.677419,...,0.790037,0.367649,0.0,1.0,1.0,0.25,0.0,0.866667,0.6,37.90
4,0.933628,0.500000,1.0,0.66061,0.0,0.0,0.833333,1.0,0.724138,0.677419,...,0.790037,0.367649,0.0,1.0,1.0,0.25,0.0,0.866667,0.6,37.28
5,0.933628,0.491803,1.0,0.66061,0.0,0.0,0.833333,1.0,0.637931,0.677419,...,0.790037,0.367649,0.0,1.0,1.0,0.25,0.0,0.866667,0.6,39.76


In [174]:
reframed.shape

(250190, 61)

In [59]:
# to split a training and test sample
from sklearn.model_selection import train_test_split, cross_val_score
train, test = train_test_split(reframed, random_state=0)

# split into input and outputs
train_X, train_y = train.drop(['speed'],1), train['speed']
test_X, test_y = test.drop(['speed'],1), test['speed']

# to values
train_X, train_y = train_X.values, train_y.values
test_X, test_y = test_X.values, test_y.values

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(187642, 1, 68) (187642,) (62548, 1, 68) (62548,)


In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# fit network
history = model.fit(train_X, train_y, epochs=25, batch_size=72, validation_data=(test_X, test_y), shuffle=False)

## LSTM Attempt 3

In [13]:
traffic_test = traffic_cnn

In [14]:
traffic_test.shape

(3000000, 34)

In [196]:
traffic_test.columns

Index(['ID', 'STATUS', 'LINK_ID', 'year', 'month', 'dayofweek', 'hour',
       'minute', 'poly_num', 'speed_id', 'speed_limit', 'lion_id', 'SeqNum',
       'StreetCode', 'StreetWidt', 'BikeLane', 'Number_Tra', 'Number_Tot',
       'weekend', 'morn_rush_hr', 'eve_rush_hr', 'morning_rush_avg_spd',
       'evening_rush_avg_spd', 'wknd_avg_spd', 'overall_avg_spd',
       'overall_std_speed', 'speed_index', 'Snow_Prior_codes', 'NonPed_codes',
       'RB_Layer_codes', 'SegmentTyp_codes', 'FeatureTyp_codes',
       'Street_codes', 'BOROUGH_codes'],
      dtype='object')

In [31]:
# to split a training and test sample
from sklearn.model_selection import train_test_split, cross_val_score
train, test = train_test_split(traffic_test, random_state=0)

# split into input and outputs
train_X, train_y = train.drop(['speed_index'],1), train['speed_index']
test_X, test_y = test.drop(['speed_index'],1), test['speed_index']
# to values
train_X, train_y = train_X.values, train_y.values
test_X, test_y = test_X.values, test_y.values
3
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

ValueError: cannot reshape array of size 74250000 into shape (2250000,2,33)

In [210]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='rmsprop', metrics=['accuracy'])
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), shuffle=False)

Train on 2250000 samples, validate on 750000 samples
Epoch 1/50
2250000/2250000 [==============================] - 66s 29us/sample - loss: 25.7389 - accuracy: 0.1668 - val_loss: 25.4970 - val_accuracy: 0.1661
Epoch 2/50
2250000/2250000 [==============================] - 64s 28us/sample - loss: 25.4302 - accuracy: 0.1668 - val_loss: 25.4975 - val_accuracy: 0.1661
Epoch 3/50
2250000/2250000 [==============================] - 66s 29us/sample - loss: 25.4302 - accuracy: 0.1668 - val_loss: 25.4970 - val_accuracy: 0.1661
Epoch 4/50
2250000/2250000 [==============================] - 62s 27us/sample - loss: 25.4302 - accuracy: 0.1668 - val_loss: 25.4969 - val_accuracy: 0.1661
Epoch 5/50
2250000/2250000 [==============================] - 57s 25us/sample - loss: 25.4302 - accuracy: 0.1668 - val_loss: 25.4975 - val_accuracy: 0.1661
Epoch 6/50
2250000/2250000 [==============================] - 57s 26us/sample - loss: 25.4302 - accuracy: 0.1668 - val_loss: 25.4973 - val_accuracy: 0.1661
Epoch 7/50


## Convolutional Neural Network

In [48]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

# set parameters:
max_features = 12
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

# to split a training and test sample
from sklearn.model_selection import train_test_split, cross_val_score
train, test = train_test_split(traffic_test, random_state=0)

# split into input and outputs
train_X, train_y = train.drop(['speed_index'],1), train['speed_index']
test_X, test_y = test.drop(['speed_index'],1), test['speed_index']
# to values
train_X, train_y = train_X.values, train_y.values
test_X, test_y = test_X.values, test_y.values

print('Pad sequences (samples x time)')
train_X = sequence.pad_sequences(train_X, maxlen=maxlen)
test_X = sequence.pad_sequences(test_X, maxlen=maxlen)
print('x_train shape:', train_X.shape)
print('x_test shape:', test_X.shape)

Pad sequences (samples x time)
x_train shape: (2250000, 400)
x_test shape: (750000, 400)


In [49]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(train_X, train_y,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_X, test_y))

Train on 2250000 samples, validate on 750000 samples
Epoch 1/2


InvalidArgumentError: indices[14,367] = 436 is not in [0, 12)
	 [[{{node embedding_2/embedding_lookup}}]] [Op:__inference_keras_scratch_graph_1481341]

## LSTM Attempt 4

In [4]:
traffic_test.head()

,Unnamed: 0,ID,STATUS,LINK_ID,year,month,dayofweek,hour,minute,poly_num,...,overall_avg_spd,overall_std_speed,speed_index,Snow_Prior_codes,NonPed_codes,RB_Layer_codes,SegmentTyp_codes,FeatureTyp_codes,Street_codes,BOROUGH_codes
0,91834,423,0,4616299,2018,2,4,0,57,44,...,46.591828,11.256594,~ Speed Limit,0,1,1,1,0,26,3
1,5139277,315,0,4616364,2018,1,4,21,13,10,...,39.032045,16.391624,~ Speed Limit,0,1,1,1,0,18,3
2,10004934,206,0,4362252,2018,10,4,21,12,53,...,40.963852,13.561896,~ Speed Limit,0,1,1,1,0,9,3
3,11320354,124,0,4456501,2018,11,5,8,43,10,...,24.835551,12.576492,~ Speed Limit,0,1,1,1,0,15,2
4,1111928,1,0,4616337,2018,3,5,7,38,13,...,13.868746,5.612008,~ Speed Limit,2,-1,0,4,0,0,2


In [17]:
traffic_test.shape

(3000000, 34)

In [36]:
X = traffic_test.drop(['speed_index'],1)
y = traffic_test['speed_index']

# to split a training and test sample
from sklearn.model_selection import train_test_split, cross_val_score
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0)

# to values
x_train, x_test, y_train, y_test = x_train.values, x_test.values, y_train.values, y_test.values

In [37]:
# reshape input to be 3D [samples, timesteps, features]
timesteps = 1

x_train = x_train.reshape((x_train.shape[0], timesteps, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], timesteps, x_test.shape[1]))
print(x_train.shape, y_train.shape, x_train.shape, y_test.shape)

(2250000, 1, 33) (2250000,) (2250000, 1, 33) (750000,)


In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

data_dim = 33
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(64, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(64, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(64))  # return a single vector of dimension 32
model.add(Dense(50, activation='softmax'))

model.compile(loss='categorical_hinge',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=120, epochs=5,
          validation_data=(x_test, y_test))

Train on 2250000 samples, validate on 750000 samples
Epoch 1/5
1718520/2250000 [=====================>........] - ETA: 20s - loss: 22.7776 - accuracy: 0.0198

KeyboardInterrupt: 